# Getting Started with XGBoost

After interpolation, SSA, and 'create_main_df_per_glacier', data should be ready for the XGBoost model (or similar machine learning model) coded below. If you're pre-processing your own data, all input variables should have data for the same dates in order to use in this model.

This code includes many plot outputs that you can comment off if you're not interested in them.

In [ ]:
# dependencies

import os
import json
import xgboost as xgb
import pandas as pd
import numpy as np
import matplotlib as mpl
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.pyplot as plt
import seaborn as sns
import ipynbname
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from scipy.signal import argrelextrema
from scipy import stats
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll.base import scope
import copy
from PIL import Image, ImageChops

import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings("ignore", message=".*The 'nopython' keyword.*")  # from shap

import shap

In [ ]:
# set font style for consistency
mpl.rcParams['font.family'] = 'Arial'  # Or any other preferred font

plt.rcParams.update({
    "axes.titlesize": 6,
    "axes.labelsize": 6,
    "xtick.labelsize": 6,
    "ytick.labelsize": 6,
    "legend.title_fontsize": 6,
    "legend.fontsize": 6
})

In [ ]:
# helper function

# objective function for hyperparameters
def objective(params):
    model = xgb.XGBRegressor(**params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    score = -r2_score(y_test, y_pred)  # Negative R2 score for minimization
    return {'loss': score, 'status': STATUS_OK}


def compute_offsets(data, predicted_col):
    offsets = []
    abs_offsets = []  # List to store absolute offsets
    
    # Ensure dates are datetime objects
    data['Date'] = pd.to_datetime(data['Date'])
    
    # Find annual max and min from observed data
    for year in data['Date'].dt.year.unique():
        year_data = data[data['Date'].dt.year == year]
        if year_data.empty:
            continue
            
        # Maxima
        data_max_idx = argrelextrema(year_data['Seasonal Terminus Position'].values, np.greater, order=2)[0]
        pred_max_idx = argrelextrema(year_data[predicted_col].values, np.greater, order=2)[0]
        offsets += calculate_nearest_offsets(year_data, data_max_idx, pred_max_idx)
        abs_offsets += [abs(offset) for offset in calculate_nearest_offsets(year_data, data_max_idx, pred_max_idx)]
        
        # Minima
        data_min_idx = argrelextrema(year_data['Seasonal Terminus Position'].values, np.less, order=2)[0]
        pred_min_idx = argrelextrema(year_data[predicted_col].values, np.less, order=2)[0]
        offsets += calculate_nearest_offsets(year_data, data_min_idx, pred_min_idx)
        abs_offsets += [abs(float(offset)) for offset in calculate_nearest_offsets(year_data, data_min_idx, pred_min_idx)]        
             
    mean_offset = np.mean(offsets) if offsets else np.nan
    mean_abs_offset = np.mean(abs_offsets) if abs_offsets else np.nan  # Mean of absolute offsets
    
    return mean_offset, mean_abs_offset


def calculate_nearest_offsets(data, data_extrema_idx, pred_extrema_idx):
    offsets = []
    if len(data_extrema_idx) == 0 or len(pred_extrema_idx) == 0:
        return offsets
    
    for data_idx in data_extrema_idx:
        data_date = data.iloc[data_idx]['Date']
        nearest_pred_idx = np.argmin(np.abs(data['Date'].iloc[pred_extrema_idx] - data_date).dt.days)
        pred_date = data.iloc[pred_extrema_idx[nearest_pred_idx]]['Date']
        offset = (pred_date - data_date).days
        offsets.append(offset)
        
    return offsets


def trim_image_whitespace(image_path, output_path=None, bgcolor=(255, 255, 255), fuzz=10):
    """
    Trims the border of an image by detecting background color (usually white).
    `fuzz` controls how tolerant the trim is to color variations.
    """
    img = Image.open(image_path).convert("RGB")

    # Create a background image of the same size with the background color
    bg = Image.new(img.mode, img.size, bgcolor)

    # Find the difference
    diff = ImageChops.difference(img, bg)

    # Increase sensitivity by converting to grayscale and amplifying
    diff = diff.convert("L")
    diff = diff.point(lambda x: 0 if x <= fuzz else 255, mode='1')

    bbox = diff.getbbox()
    if bbox:
        cropped_img = img.crop(bbox)
        save_path = output_path or image_path
        cropped_img.save(save_path)
        return save_path
    else:
        return image_path  # no trimming needed

In [ ]:
# open glacierid file
file_path = 'processing_scripts/qualifying_glacierids.json'
with open(file_path, 'r') as file:
    glacier_ids = json.load(file)

In [ ]:
# main iteration
for glacierid in glacier_ids:
    glacierid = glacierid
    print('Working on glacier: '+glacierid)
    filepath = ".../all_features_per_glacier/"
    file_path = os.path.join(filepath, f"{glacierid}_main_df.csv")
    data = pd.read_csv(file_path, header=0)
    y = data['Seasonal Terminus Position']  # target variable
    # drop any unused variables from your df
    X = data.drop(columns=['Date', 'Seasonal Terminus Position', 'Seasonal Discharge-RACMO', 'Seasonal OTF-ECCO'])
    current_column_names = X.columns
    # rename variable names if desired
    new_column_names = ['Air Temperature','Runoff','Velocity',
                       'Strain Rate','Surface Elevation','Bed Elevation',
                       'Thickness','Slope','Melange','Ocean Thermal Forcing']
    column_rename_dict = dict(zip(current_column_names, new_column_names))
    X.rename(columns=column_rename_dict, inplace=True)
    # set train/test data split; test_size=0.2 for 80/20 split.
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
    # Define the search space for hyperparameters
    param_space = {
        'tree_method': hp.choice('tree_method', ['approx']),  
        'objective': hp.choice('objective', ['reg:squarederror']),  
        'n_estimators': scope.int(hp.quniform('n_estimators', 50, 300, 10)),
        'max_depth': scope.int(hp.randint('max_depth', 2, 10)),  # Convert to int explicitly
        'learning_rate': hp.uniform('learning_rate', 0.01, 0.2),
        'alpha': hp.uniform('alpha', 0, 100), 
        'reg_lambda': hp.uniform('reg_lambda', 1, 20), 
        'max_delta_step': scope.int(hp.randint('max_delta_step', 0, 50)),  # Convert to int explicitly
        'colsample_bytree': hp.uniform('colsample_bytree', 0.2, 1.0) 
    }

    trials = Trials()
    best = fmin(fn=objective,
                space=param_space,
                algo=tpe.suggest,
                max_evals=50,
                trials=trials,
                rstate=np.random.default_rng(42))

    best_params = {
        'tree_method': 'approx',
        'objective': 'reg:squarederror',
        'n_estimators': int(best['n_estimators']),
        'max_depth': int(best['max_depth']),
        'learning_rate': best['learning_rate'],
        'alpha': best['alpha'],
        'reg_lambda': best['reg_lambda'],
        'max_delta_step': best['max_delta_step'],
        'colsample_bytree': best['colsample_bytree']
    }
    
    # Define evaluation dataset
    evalset = [(X_train, y_train), (X_test, y_test)]
    final_model = xgb.XGBRegressor(**best_params,random_state=42)
    final_model.fit(
        X_train, y_train,
        eval_set=evalset,
        verbose=False
    )
    pred_train = final_model.predict(X_train)
    pred_test = final_model.predict(X_test)
    train_r2 = r2_score(y_train, pred_train)
    test_r2 = r2_score(y_test, pred_test)
    results = final_model.evals_result()

    # Plot learning curves
    plt.figure(figsize=(10, 6))
    plt.plot(results['validation_0']['rmse'], label='Train')
    plt.plot(results['validation_1']['rmse'], label='Test')
    plt.legend()
    plt.title('Root Mean Squared Error')
    output_dir = os.path.join(os.getcwd(), 'outputs', 'RMSE_plot')
    os.makedirs(output_dir, exist_ok=True)
    file_path = os.path.join(output_dir, f"{glacierid}_rmse_plot.png")
    plt.savefig(file_path)
    plt.clf()  
    plt.cla() 
    plt.close()
    
    # Predict Seasonal Comp
    data['Predicted Seasonal Comp'] = final_model.predict(X)
    data3 = pd.DataFrame()
    data3['Predicted Seasonal Comp'] = final_model.predict(X_train)
    train_rmse = mean_absolute_error(y_train.values, data3['Predicted Seasonal Comp'].values)
    data2 = pd.DataFrame()
    data2['Predicted Seasonal Comp'] = final_model.predict(X_test)
    test_rmse = mean_absolute_error(y_test.values, data2['Predicted Seasonal Comp'].values)
    
    # Normalized RMSE using average max minus average min of testing data
    avg_max = np.mean([np.max(y_test.values[i::1]) for i in range(1)])  # Assuming annual cycles
    avg_min = np.mean([np.min(y_test.values[i::1]) for i in range(1)])
    normalization_factor = avg_max - avg_min

    train_nrmse = train_rmse / normalization_factor
    test_nrmse = test_rmse / normalization_factor
    
    # add the Spearman correlation coefficient
    res = stats.spearmanr(y_test, pred_test)
    spearman_stat = res.statistic
    spearman_p    = res.pvalue
    
    # add R2 computation
    test_r2 = r2_score(y_test, pred_test)
    
    new_entry = pd.DataFrame({
        'glacierid'         : [glacierid],
        'Test RMSE'         : [test_rmse],
        'Test NRMSE'        : [test_nrmse],
        'Spearman Statistic': [spearman_stat],
        'Spearman P-Value'  : [spearman_p],
        'Test R2 Score'     : [test_r2]
    })
    
    
    # save the output
    output_file = os.path.join(os.getcwd(), 'outputs', 'error_scores_nrmse.csv')
    if os.path.exists(output_file):
        existing_data = pd.read_csv(output_file)
        updated_data = pd.concat([existing_data, new_entry], ignore_index=True)
        updated_data['glacierid'] = updated_data['glacierid'].astype(int).astype(str).str.zfill(3)
        updated_data = updated_data.drop_duplicates(subset=['glacierid'])
        updated_data = updated_data.sort_values(by='glacierid')
    else:
        updated_data = new_entry
    os.makedirs(os.path.dirname(output_file), exist_ok=True)
    updated_data.to_csv(output_file, index=False)
    
    # Compute offsets and absolute offsets
    mean_offset, mean_abs_offset = compute_offsets(data, 'Predicted Seasonal Comp')
    offset_entry = pd.DataFrame({
        'glacierid': [glacierid],
        'mean offset': [float(mean_offset)],  
        'absolute offset': [float(mean_abs_offset)] 
    })

    offset_file = os.path.join(os.getcwd(), 'outputs', 'error_scores_offset.csv')
    if os.path.exists(offset_file):
        existing_offset = pd.read_csv(offset_file)
        updated_offset = pd.concat([existing_offset, offset_entry], ignore_index=True)
        updated_offset['glacierid'] = updated_offset['glacierid'].astype(int).astype(str).str.zfill(3)
        updated_offset = updated_offset.drop_duplicates(subset=['glacierid'], keep='last')
    else:
        updated_offset = offset_entry

    os.makedirs(os.path.dirname(offset_file), exist_ok=True)
    updated_offset.to_csv(offset_file, index=False)

    # Save prediction plot
    data['Date'] = pd.to_datetime(data['Date'])
    predicted_data = data[['Date', 'Predicted Seasonal Comp']].copy()
    test_indices = X_test.index
    test_period_start = data.loc[test_indices, 'Date'].min()
    test_period_end = data.loc[test_indices, 'Date'].max()
    predicted_data_test = predicted_data[(predicted_data['Date'] >= test_period_start) & 
                                         (predicted_data['Date'] <= test_period_end)]
    plt.figure(figsize=(1.9, 1.9))
    plt.subplots_adjust(left=0.13, right=0.95)  
    plt.plot(data['Date'], data['Seasonal Terminus Position'], color='green', linewidth=1, label='Actual')
    plt.plot(predicted_data_test['Date'], predicted_data_test['Predicted Seasonal Comp'],
             color='black', alpha=0.6, linewidth=1, label='Model')
    plt.axvspan(test_period_start, test_period_end, facecolor='green', alpha=0.1)
    plt.ylabel('Terminus Advance (m)', labelpad=2.5)  
    ax = plt.gca()
    ax.tick_params(axis='y', pad=2)  
    plt.legend(loc='upper left', ncol=2, markerscale=0.7, handletextpad=0.2, borderpad=0.10, labelspacing=2)

    output_dir = os.path.join(os.getcwd(), 'outputs', 'prediction_plot')
    os.makedirs(output_dir, exist_ok=True)
    output_file = os.path.join(output_dir, f'{glacierid}_prediction.plot.png')
    plt.savefig(output_file, dpi=300,
                format='png', metadata={'Creator': 'Matplotlib'},
               bbox_inches='tight')
    plt.clf()  
    plt.cla()  
    plt.close()
    trim_image_whitespace(output_file, bgcolor=(255, 255, 255), fuzz=10)

    
    # Create an explainer for the model using SHAP
    explainer = shap.Explainer(final_model.predict, X)
    shap_test = explainer(X_test)
    
    # Define the new feature names for figures
    abs_feature_names = [
        'Air Temp.', 'Runoff', 'Velocity', 'Strain Rate', 
        'Surface Ele.', 'Bed Ele.', 'Thickness', 
        'Bed Slope', 'Mélange', 'OTF'
    ]
    rel_feature_names = [
        'Air Temp. (°C)', 'Runoff (m³/s)', 'Velocity (m/yr)', 'Strain R. (yr$^{-1}$)', 
        'Surface Ele. (m)', 'Bed Ele. (m)', 'Thickness (m)', 
        'Bed Slope (m)', 'Mélange (°C)', 'OTF (°C)'
    ]

    abs_shap_test = copy.deepcopy(shap_test)
    rel_shap_test = copy.deepcopy(shap_test)
    abs_shap_test.feature_names = abs_feature_names
    rel_shap_test.feature_names = rel_feature_names
    
    # SHAP values - absolute names
    mean_abs_shap_values = np.abs(abs_shap_test.values).mean(axis=0)
    sorted_feature_indices = np.argsort(mean_abs_shap_values)[::-1]
    sorted_feature_names = [abs_shap_test.feature_names[i] for i in sorted_feature_indices]
    shap_df = pd.DataFrame(abs_shap_test.values, columns=abs_shap_test.feature_names, index=X_test.index)
    shap_df = shap_df[sorted_feature_names]
    test_period_start = data.loc[X_test.index.min(), 'Date']
    test_period_end = data.loc[X_test.index.max(), 'Date']
    test_dates = data[(data['Date'] >= test_period_start) & (data['Date'] <= test_period_end)]['Date']
    shap_df.index = test_dates.values
    shap_values_dir = "/Users/kevin/Documents/ML_longterm/XGBoost/seasonal/full_features/outputs/shap_values"
    os.makedirs(shap_values_dir, exist_ok=True)
    shap_values_filename = os.path.join(shap_values_dir, f"{glacierid}_shap_values.csv")
    shap_df.to_csv(shap_values_filename)

    # SHAP values - relationship names
    rel_mean_abs_shap_values = np.abs(rel_shap_test.values).mean(axis=0)
    rel_sorted_feature_indices = np.argsort(rel_mean_abs_shap_values)[::-1]
    rel_sorted_feature_names = [rel_shap_test.feature_names[i] for i in rel_sorted_feature_indices]
    rel_shap_df = pd.DataFrame(rel_shap_test.values, columns=rel_shap_test.feature_names, index=X_test.index)
    rel_shap_df = rel_shap_df[rel_sorted_feature_names]
    test_period_start = data.loc[X_test.index.min(), 'Date']
    test_period_end = data.loc[X_test.index.max(), 'Date']
    test_dates = data[(data['Date'] >= test_period_start) & (data['Date'] <= test_period_end)]['Date']
    rel_shap_df.index = test_dates.values

    # SHAP Percentages
    mean_absolute_values = shap_df.abs().mean()
    total_sum = mean_absolute_values.sum()
    ranking_df = (mean_absolute_values / total_sum * 100).round(2)
    ranking_df = ranking_df.to_frame().T
    ranking_df.insert(0, 'glacierid', glacierid)
    shap_percentages_file = "/Users/.../outputs/shap_percentages.csv"
    if os.path.exists(shap_percentages_file):
        existing_df = pd.read_csv(shap_percentages_file)
        existing_df = pd.concat([existing_df, ranking_df], ignore_index=True)
        existing_df['glacierid'] = existing_df['glacierid'].astype(int).astype(str).str.zfill(3)
        existing_df = existing_df.drop_duplicates(subset=['glacierid'], keep='last')
        existing_df = existing_df.sort_values(by='glacierid')
    else:
        existing_df = ranking_df
    existing_df.to_csv(shap_percentages_file, index=False)
    
    # SHAP Mean Plot
    mean_shap_dir = "/Users/.../outputs/shap_mean_plots"
    summary_shap_dir = "/.../outputs/shap_summary_plots"
    os.makedirs(mean_shap_dir, exist_ok=True)
    os.makedirs(summary_shap_dir, exist_ok=True)
    columns = shap_df.abs().mean().sort_values(ascending=False).index
    feature_categories = {
        'Bed Ele.': ('Geometric', 'tab:blue'),
        'Surface Ele.': ('Geometric', 'tab:blue'),
        'Bed Slope': ('Geometric', 'tab:blue'),
        'Thickness': ('Geometric', 'tab:blue'),
        'Velocity': ('Dynamic', '#FFD700'),
        'Strain Rate': ('Dynamic', '#FFD700'),
        'OTF': ('Climatic', 'tab:red'),
        'Air Temp.': ('Climatic', 'tab:red'),
        'Runoff': ('Climatic', 'tab:red'),
        'Mélange': ('Climatic', 'tab:red'),
    }
    legend_dict = {}
    legend_colors = []
    labels = []
    for column in columns:
        category, color = feature_categories.get(column, ('Unknown', 'gray'))
        if category not in legend_dict:
            legend_dict[category] = color
        labels.append(category)
        legend_colors.append(color)
    
    fig, ax = plt.subplots(figsize=(1.1, 1.8))
    plt.subplots_adjust(left=0.05)  
    sns.barplot(x=shap_df[columns].abs().mean(), y=columns, palette=legend_colors, ax=ax)
    ax.set_title("")  # Remove the title
    ax.set_xlabel("Absolute SHAP Value")  
    ax.set_ylabel("") 
    handles = [plt.Line2D([0], [0], marker='o', color='w', markerfacecolor=color, markersize=6) for color in legend_dict.values()]
    ax.legend(handles, legend_dict.keys(), 
        loc='lower right', 
        handletextpad=0.3, borderpad=0.15,labelspacing=0.2
    )
    mean_shap_filepath = os.path.join(mean_shap_dir, f"{glacierid}_mean_shap.png")
    plt.savefig(mean_shap_filepath, dpi=300, bbox_inches='tight')
    plt.clf()  
    plt.cla()  
    plt.close(fig)
    trim_image_whitespace(mean_shap_filepath, bgcolor=(255, 255, 255), fuzz=10)

    
    # SHAP Summary Plot
    fig_summary, ax_summary = plt.subplots()
    shap.summary_plot(shap_test, show=False)
    summary_shap_filepath = os.path.join(summary_shap_dir, f"{glacierid}_shap_summary_plot.png")
    plt.savefig(summary_shap_filepath, dpi=300, format='png', metadata={'Creator': 'Matplotlib'})
    plt.clf()  # Clear the figure
    plt.cla()  # Clear the axes
    plt.close(fig_summary)

    # SHAP Relationshp plot (seasonal color version)
    rename_dict = {
        'Air Temperature': 'Air Temp. (°C)',
        'Runoff': 'Runoff (m³/s)',
        'Velocity': 'Velocity (m/yr)',
        'Strain Rate': 'Strain R. (yr$^{-1}$)',
        'Surface Elevation': 'Surface Ele. (m)',
        'Bed Elevation': 'Bed Ele. (m)',
        'Thickness': 'Thickness (m)',
        'Slope': 'Bed Slope (m)',
        'Melange': 'Mélange (°C)',
        'Ocean Thermal Forcing': 'OTF (°C)'
    }
    X_test = X_test.rename(columns=rename_dict)
    variables = rel_shap_df.columns
    nrows, ncols = 2, 5
    
    subplot_width = 2.5  
    subplot_height = 4.9   
    fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(5.9,3), sharey='row')
    day_of_year = rel_shap_df.index.dayofyear
    day_of_year_normalized = day_of_year / 365.0
    cmap = plt.cm.twilight
    colors = cmap(day_of_year_normalized)
    
    for i, variable in enumerate(variables):
        ax = axes[i // ncols, i % ncols]
        shap_values = rel_shap_df[variable].values
        feature_values = X_test[variable].values
        scatter = ax.scatter(
            feature_values,
            shap_values,
            c=day_of_year_normalized,
            s=0.2,
            cmap=cmap,
        )
        ax.set_title(variable, y=-.5) 
    
        # Reduce tick label padding
        ax.tick_params(axis='x', pad=2)
        if abs(feature_values.max() - feature_values.min()) < 0.01:
            ax.xaxis.set_major_locator(plt.MaxNLocator(nbins=3))
            ax.ticklabel_format(axis='x', style='plain', useOffset=False)

        # strain rate label customization
        if variable == 'Strain R. (yr$^{-1}$)':
            xticks = ax.get_xticks()
            if np.any(np.abs(xticks) < 0.1) and np.any(np.abs(xticks) > 0):  
                ax.ticklabel_format(axis='x', style='sci', scilimits=(-2, 2))  
                for label in ax.get_xticklabels():
                    label.set_verticalalignment('top')  
                    label.set_y(0.25)
    
        if i % ncols == 0:
            ax.set_ylabel("SHAP value")
            ax.set_yticks(range(-100, 101, 50))  # fewer ticks
        else:
            ax.get_yaxis().set_visible(False)
            ax.spines['left'].set_visible(False)
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        ax.set_ylim(-100, 100)
    
    # Positioning the colorbar to the right of all plots
    cbar_ax = fig.add_axes([0.88, 0.15, 0.02, 0.7]) 
    cbar = fig.colorbar(scatter, cax=cbar_ax)
    season_locs = [5, 128, 255, 383, 510]
    season_labels = ['Winter', 'Spring', 'Summer', 'Fall', 'Winter']
    cbar.set_ticks(season_locs)
    cbar.set_ticklabels(season_labels)
    plt.subplots_adjust(left=0.1, right=0.85, top=0.85, bottom=0.15, wspace=0.3, hspace=0.5)
    
    relationships_plot_dir = "/Users/kevin/Documents/ML_longterm/XGBoost/seasonal/full_features/outputs/shap_relationships_plot"
    os.makedirs(relationships_plot_dir, exist_ok=True)
    relationships_plot_filepath = os.path.join(relationships_plot_dir, f"{glacierid}_relationships_plot.png")
    plt.savefig(relationships_plot_filepath, dpi=300, bbox_inches='tight', format='png', metadata={'Creator': 'Matplotlib'})
    plt.clf()  
    plt.cla()  
    plt.close(fig)